In [11]:
# 형태소 분석/텍스트 클리닝
from nltk.corpus import stopwords
import nltk
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
from nltk import word_tokenize
import pandas as pd 
from tqdm import tqdm
from konlpy.tag import Okt
import re

# 단어 추가를 위함
from ckonlpy.tag import Twitter


### konlpy.tag import Okt에 단어 추가하기
- https://inspiringpeople.github.io/data%20analysis/ckonlpy/

In [31]:
pip install customized_konlpy

Name: konlpy
Version: 0.6.0
Summary: Python package for Korean natural language processing.
Home-page: http://konlpy.org
Author: Team KoNLPy
Author-email: konlpy@googlegroups.com
License: GPL v3
Location: c:\users\user\anaconda3\lib\site-packages
Requires: JPype1, lxml, numpy
Required-by: customized-konlpy
Note: you may need to restart the kernel to use updated packages.


In [36]:
## 테스트
twitter = Twitter()

print(twitter.nouns('문제해결능력'))
twitter.add_dictionary(['문제해결능력','DBMS'], 'Noun')

print(twitter.nouns('문제해결능력'))
print(twitter.nouns('DBMS'))

['문제해결', '능력']
['문제해결능력']
['DBMS']


### 데이터 불러오기

In [8]:
df = pd.read_csv('기업데이터(30)_(전처리).csv').iloc[:, 1:]
df.columns = ["com", "date", "duty", "status", "star", "summary", "good", "bad", "expect", "doc"]
df.head()
df1 = df[["com", "date", "duty", "doc"]]

In [12]:
df1.head(1)

,com,date,duty,doc
0,네이버,2022. 11,디자인,커리어 경력 쌓고 싶은 사람에게 추천 수평적 사무실 분위기와 복지가 다른 곳 보다는...


### 불용어, 숫자, 띄어쓰기, 특수문자, 의성어, 이모티콘 Cleaning
- doc -> processing

In [13]:


def clean_text(texts):
    #이모티콘 제거
    emoji_pattern = re.compile("["
            u"\U0001F600-\U0001F64F"  # emoticons
            u"\U0001F300-\U0001F5FF"  # symbols & pictographs
            u"\U0001F680-\U0001F6FF"  # transport & map symbols
            u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                            "]+", flags=re.UNICODE)

    #분석에 어긋나는 불용어구 제외 (특수문자, 의성어)
    han = re.compile(r'[ㄱ-ㅎㅏ-ㅣ!?~,".\n\r#\ufeff\u200d]')
    
    corpus = []
    for i in range(0, len(texts)):
        review = re.sub(r'[@%\\*=()/~#&\+á?\xc3\xa1\-\|\.\:\;\!\-\,\_\~\$\'\"]', '',str(texts[i])) #remove punctuation
        review = re.sub(r'\d+','', str(texts[i]))# remove number
        review = review.lower() #lower case
        review = re.sub(r'\s+', ' ', review) #remove extra space
        # review = re.sub(r'<[^>]+>','',review) #remove Html tags
        review = re.sub(r'\s+', ' ', review) #remove spaces
        review = re.sub(r"^\s+", '', review) #remove space from start
        review = re.sub(r'\s+$', '', review) #remove space from the end
        review = re.sub(han, '', review) #remove 특수문자, 의성어
        review = re.sub(emoji_pattern, '', review) #remove 이모티콘
        corpus.append(review)
    return corpus

df1["processing"] = clean_text(df1["doc"])

In [15]:
df1["processing"] = clean_text(df1["doc"])

C:\Users\user\AppData\Local\Temp\ipykernel_16048\3180551206.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1["processing"] = clean_text(df1["doc"])


In [16]:
df1.head()

,com,date,duty,doc,processing
0,네이버,2022. 11,디자인,커리어 경력 쌓고 싶은 사람에게 추천 수평적 사무실 분위기와 복지가 다른 곳 보다는...,커리어 경력 쌓고 싶은 사람에게 추천 수평적 사무실 분위기와 복지가 다른 곳 보다는...
1,네이버,2022. 11,전문직,자유로운 복장 분위기가 일단 편해서 좋았어요. 물론 업무는 당연히 강도가 있어야 할...,자유로운 복장 분위기가 일단 편해서 좋았어요 물론 업무는 당연히 강도가 있어야 할 ...
2,네이버,2022. 11,IT/인터넷,워라밸과 성장을 동시에 챙길 수 있는 몇 안되는 기업 앞으로도 가장 전망이 좋은 플...,워라밸과 성장을 동시에 챙길 수 있는 몇 안되는 기업 앞으로도 가장 전망이 좋은 플...
3,네이버,2022. 11,IT/인터넷,개발자가 영향력을 좀 발휘하며 일할수 있는곳. it업계 1위 개발에만 집중하며 일할...,개발자가 영향력을 좀 발휘하며 일할수 있는곳 it업계 위 개발에만 집중하며 일할 수...
4,네이버,2022. 10,IT/인터넷,아르바이트 생이었지만 꿈의 직장이란게 이런 것이구나 느낄 수 있었다 휴가 연차 등 ...,아르바이트 생이었지만 꿈의 직장이란게 이런 것이구나 느낄 수 있었다 휴가 연차 등 ...


### 맞춤법검사기

In [42]:
pip install git+https://github.com/ssut/py-hanspell.git

In [43]:
from hanspell import spell_checker

sent = "맞춤법 틀리면 외 않되? 쓰고싶은대로쓰면돼지 "
spelled_sent = spell_checker.check(sent)

hanspell_sent = spelled_sent.checked
print(hanspell_sent)

맞춤법 틀리면 왜 안돼? 쓰고 싶은 대로 쓰면 되지


In [44]:
spelled_sent = spell_checker.check("아르바이트 생이었지만 꿈의 직장이란게 이런 것이구나 느낄 수 있었다")
hanspell_sent = spelled_sent.checked
print(hanspell_sent)

pip install soynlp

아르바이트 생이었지만 꿈의 직장이란 게 이런 것이구나 느낄 수 있었다


### 형태소별 분리 및 불용어(은, 는, 이, 가 등) 제거
- processing -> token

In [37]:
twitter = Twitter()

# 필요한 단어 추가
twitter.add_dictionary(['워라벨','워라밸', '주말근무', '경영진', '적극', '인상', '전문', ], 'Noun')
twitter.add_dictionary(['새로운'], 'Adjective')

 # 한국어 불용어 사전 불러오기
 # 의미가 없거나 없어도 되는 단어를 모아놓은 파일
stopwords = pd.read_csv('ko-stopwords_형태소분석_수업용.csv'); print(stopwords[0:3])  
# 리스트로 만들어줌
stopwords = list(stopwords['stopwords']); print(stopwords[0:3])
# 크롤링한 내용을 보면서 불용어를 추가로 넣어줌(extend)
stopwords.extend(['에서','고','이다','는','한','씨', "것","거","게","데","이다","건","고","되다","되어다","걸","기",
                  "시","네","듯","랍니","중이","얘","스","도도", "나","수","개","내","기","제","저","인","있다","이렇다",
                  "그렇다","번","위","팅","분","인","링","란","포","두", "진짜", "하다" ,"이다" ,"가다", "이제" ,"들다"
                 ]) 
# 혹시 중복이 되어 있을 수 있어 중복제거 해줌
stopwords = set(stopwords) 

# 형태소 분석 Function
def okt_pos_tagging(string):
    # pos는 주어진 텍스트를 형태소 단위로 나누고 품사를 태깅한다.(형태소, 품사)
    # 예: [('유일하', 'Noun'), ('게', 'Josa'), ('되다', 'Verb')]
    pos_words = twitter.pos(string, stem= True, norm = True) 
    words = [word for word, tag in pos_words if tag
             in ['Noun', 'Adjective', 'Verb', ' Adverb'] if word not in stopwords]
    return words    
    
 

  stopwords
0         가
1      가까스로
2        가령
['가', '가까스로', '가령']


In [38]:
# 형태소 분석 적용
tokenizing_doc = []
for i in tqdm(df1['processing']):
    tokenizing_doc.append(okt_pos_tagging(i))
df1['token'] = tokenizing_doc #새로운 tokenizing이라는 컬럼에 키워드 형태로 만든 원리
df1[["com", "date", "duty", "processing", "token"]]

100%|██████████| 60573/60573 [07:16<00:00, 138.63it/s]
C:\Users\user\AppData\Local\Temp\ipykernel_16048\514626234.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['token'] = tokenizing_doc #새로운 tokenizing이라는 컬럼에 키워드 형태로 만든 원리


,com,date,duty,processing,token
0,네이버,2022. 11,디자인,커리어 경력 쌓고 싶은 사람에게 추천 수평적 사무실 분위기와 복지가 다른 곳 보다는...,"[커리어, 경력, 쌓다, 싶다, 사람, 추천, 수평, 적, 사무실, 분위기, 복지,..."
1,네이버,2022. 11,전문직,자유로운 복장 분위기가 일단 편해서 좋았어요 물론 업무는 당연히 강도가 있어야 할 ...,"[자유로, 운, 복장, 분위기, 편, 좋다, 어요, 업무, 강도, 부분, 챙기다, ..."
2,네이버,2022. 11,IT/인터넷,워라밸과 성장을 동시에 챙길 수 있는 몇 안되는 기업 앞으로도 가장 전망이 좋은 플...,"[워라밸, 성장, 동시, 챙기다, 안되다, 기업, 앞, 가장, 전망, 좋다, 플랫폼..."
3,네이버,2022. 11,IT/인터넷,개발자가 영향력을 좀 발휘하며 일할수 있는곳 it업계 위 개발에만 집중하며 일할 수...,"[개발자, 영향력, 발, 휘하, 곳, 업계, 개발, 집, 중하, 보상, 복지, 좋다..."
4,네이버,2022. 10,IT/인터넷,아르바이트 생이었지만 꿈의 직장이란게 이런 것이구나 느낄 수 있었다 휴가 연차 등 ...,"[아르바이트, 생, 꿈, 직장, 이란, 이구, 느끼다, 휴가, 연차, 복지, 눈치,..."
...,...,...,...,...,...
60568,여기어때,2014. 12,IT/인터넷,돈은 있다 성장을 원한다 높은 보상을 원한다면 도전해보라 조직원 평균연령이 젊음(그...,"[돈, 성장, 원한, 높다, 보상, 원한, 면, 도, 전해, 보라, 조직, 원, 평..."
60569,여기어때,2014. 11,IT/인터넷,자율적인 분위기 깨어있는 조직문화 안정적인 회사를 원하면 지원해도됨 경영진의 마인드...,"[자율, 적, 분위기, 깨다, 조직, 문화, 안, 정적, 회사, 원하다, 지원, 해..."
60570,여기어때,2014. 11,IT/인터넷,best 일에 대한 열정이 있고 대인관계에 자신이 있는 자는 쉽게 높은 연봉과 직급...,"[대한, 열정, 대인관계, 자다, 쉬다, 높다, 연봉, 직급, 얻다, 젊다, 분위기..."
60571,여기어때,2014. 10,IT/인터넷,젊은 분위기 야근 많고 주말근무도 많음 월급은 적음 작은 규모이지만 비전을 가지고 ...,"[젊다, 분위기, 야근, 많다, 주말, 근무, 많다, 월급, 적다, 작다, 규모, ..."


In [40]:
df1[["processing",'token']].to_csv("단어추가_전_토큰화csv", encoding='utf-8-sig', index=False)

In [ ]:
# 저장
data.to_excel('블로그_비건(전처리)_수업용.xlsx', encoding='utf-8-sig', index=False)

In [41]:
kkk = twitter.pos("새로운", stem= True, norm = True) 
kkk

[('새', 'Noun'), ('로운', 'Josa')]